<a href="https://colab.research.google.com/github/Adi1116/Accepting-or-Rejecting-a-Sensor/blob/main/MindCase_AdityaSharma_Assignmnent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pypdf
!pip install torch
!pip install -q transformers
!pip -q install sentence-transformers
!pip install -q llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 192.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 251.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.62-cp310-cp310-manylinux_2_35_x86_64.whl size=38976636 sha256=f0af59a1dc5bf523f6235a21068cb66c47a2adb5e0f508dbe1296ff93901ecde
  Stored in directory: /tmp/pip-ephem-wheel-cache-g1l5fr60/wheels/c0/81/de/d4cc8f152d89865379dbf28ca672358c667192ee55deaca7cb
Successfully built llama-cpp-python


In [3]:
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-llama-cpp

In [4]:
!pip install cuda

ERROR: Could not find a version that satisfies the requirement cuda (from versions: none)
ERROR: No matching distribution found for cuda


In [5]:
import torch
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    # model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

total size (MB): 4368.44


4167it [00:23, 180.16it/s]                          
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension

In [6]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Document

documents = SimpleDirectoryReader(
    input_files = ["/content/Mindcase Data.pdf"]
).load_data()

documents = Document(text = "\n\n".join([doc.text for doc in documents]))

In [7]:
import os
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import VectorStoreIndex, ServiceContext, load_index_from_storage
def get_build_index(documents,llm,embed_model="local:BAAI/bge-small-en-v1.5",sentence_window_size=3,save_dir="./vector_store/index"):

  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

  sentence_context = ServiceContext.from_defaults(
      llm = llm,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  if not os.path.exists(save_dir):
        # create and load the index
        index = VectorStoreIndex.from_documents(
            [documents], service_context=sentence_context
        )
        index.storage_context.persist(persist_dir=save_dir)
  else:
      # load the existing index
      index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
      )

  return index

In [8]:
# get the vector index
vector_index = get_build_index(documents=documents, llm=llm, embed_model="local:BAAI/bge-small-en-v1.5", sentence_window_size=3, save_dir="./vector_store/index")

<ipython-input-7-c06f19ff2935>:12: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )
  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine

In [10]:
query_engine = get_query_engine(sentence_index=vector_index, similarity_top_k=6, rerank_top_n=2)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [11]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)
  print("\n")

How many male and female characters are in the movie?



llama_print_timings:        load time =     618.18 ms
llama_print_timings:      sample time =      56.27 ms /   101 runs   (    0.56 ms per token,  1794.85 tokens per second)
llama_print_timings: prompt eval time =     617.56 ms /   395 tokens (    1.56 ms per token,   639.62 tokens per second)
llama_print_timings:        eval time =    2420.94 ms /   100 runs   (   24.21 ms per token,    41.31 tokens per second)
llama_print_timings:       total time =    3412.13 ms /   495 tokens


 Based on the provided context, there are at least eight characters mentioned: K (male), Doxie #1, Doxie #2 (female), Doxie #3 (female), Mariette (female), Punk Doxie (Mariette) (female), The Emmanator (male), and Sapper Morton (male). Therefore, there are five female characters and three male characters in the movie according to this context.


Does the script pass the Bechdel test?


Llama.generate: prefix-match hit

llama_print_timings:        load time =     618.18 ms
llama_print_timings:      sample time =      41.75 ms /    72 runs   (    0.58 ms per token,  1724.67 tokens per second)
llama_print_timings: prompt eval time =     380.46 ms /   268 tokens (    1.42 ms per token,   704.42 tokens per second)
llama_print_timings:        eval time =    1729.45 ms /    71 runs   (   24.36 ms per token,    41.05 tokens per second)
llama_print_timings:       total time =    2366.18 ms /   339 tokens


 Based on the provided context, the script does not explicitly pass the Bechdel test as there is no conversation between two female characters about something other than a man. However, it's important to note that the Bechdel test is just one way to evaluate gender representation in media and doesn't capture the full complexity of character interactions or relationships.


Explain the theme of the movie?


Llama.generate: prefix-match hit

llama_print_timings:        load time =     618.18 ms
llama_print_timings:      sample time =     127.64 ms /   209 runs   (    0.61 ms per token,  1637.47 tokens per second)
llama_print_timings: prompt eval time =     378.25 ms /   304 tokens (    1.24 ms per token,   803.71 tokens per second)
llama_print_timings:        eval time =    5121.09 ms /   208 runs   (   24.62 ms per token,    40.62 tokens per second)
llama_print_timings:       total time =    6314.88 ms /   512 tokens


 Based on the provided context, it appears that the movie explores themes of identity, deception, and morality. The characters in the scene are discussing a past event where K, the main character, had made an error in identifying a target, leading to unintended consequences. Freysa seems to suggest that this incident is just a piece of a larger puzzle, implying that there may be more complex truths to uncover. The revelation of this mistake causes K to experience a wave of nausea and a crushing realization that he has been living with a false sense of self. In another scene, Ana recognizes something in K's memory, suggesting a connection between their pasts. The movie seems to be questioning the nature of truth, identity, and the consequences of our actions. Additionally, there is a reference to a casino called "The Casinoint" and a device called "Luv's Spinner," but without further context, it is unclear how these elements fit into the overall theme.




KeyboardInterrupt: Interrupted by user